We want to solve:
$$0=h(k)-(u')^{-1}\left( \beta u'\left( h\left((f(k) +(1- \delta)k- h(k)\right) \right) f'\left(f(k) +(1- \delta)k - h(k) \right) \right)$$
where $c=h(k)$ using projection on Chebychev polynomials.

In [20]:
using Parameters
using ChebyshevApprox

In [2]:
@with_kw immutable GrowthModel
    β::Float64                           # discount factor
    δ::Float64                           # depreciation
    α::Float64                           # capital share
    A::Float64                           # productivity
    γ::Float64                           # RRA
end

model = GrowthModel(β = 0.95, δ = 0.05, α = 0.3, A = 1, γ = 2)

function steady_state_k(model::GrowthModel)
    @unpack β, α, A, δ = model
    ((1/β-(1-δ))/(A*α))^(1/(α-1))
end

function f(model::GrowthModel, k)
    @unpack α, A, δ = model
    A*k^α
end

function f_prime(model::GrowthModel, k)
    @unpack α, A, δ = model
    A*α*k^(α-1)
end

u_crra_prime(c, γ) = c^-γ
u_crra_prime_inv(u, γ) = u^(-1/γ)

u_crra_prime_inv (generic function with 1 method)

In [22]:
k_stst = steady_state_k(model)

4.628988089138438

In [14]:
function Y(model::GrowthModel, k, a)
    @unpack β, α, A, δ, γ = model
    
    u_crra_prime_inv(β*u_crra_prime(h(f(model,k) + (1-δ)*k - h(k)), γ) * 
            (f_prime(model, f(model,k) + (1-δ)*k - h(k)) + 1-δ), γ)
end

euler_error (generic function with 1 method)

In [24]:
domain = [0.1*k_stst, 8*k_stst]
nodes = chebyshev_nodes(20, domain)


20-element Array{Float64,1}:
 36.9755  
 36.5267  
 35.6401  
 34.3375  
 32.651   
 30.6222  
 28.301   
 25.7446  
 23.0158  
 20.182   
 17.3128  
 14.479   
 11.7502  
  9.19378 
  6.87257 
  4.84376 
  3.1573  
  1.85472 
  0.968101
  0.519264

In [26]:
using QuantEcon

INFO: Precompiling module QuantEcon.
ERROR: LoadError: LoadError: LoadError: Unable to load 

libhdf5 (/Users/Max/.julia/v0.5/Homebrew/deps/usr/lib/libhdf5.dylib)

Please re-run Pkg.build(package), and restart Julia.
 in include_from_node1(::String) at ./loading.jl:488
 in include_from_node1(::String) at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 in include_from_node1(::String) at ./loading.jl:488
 in include_from_node1(::String) at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 in include_from_node1(::String) at ./loading.jl:488
 in include_from_node1(::String) at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 in macro expansion; at ./none:2 [inlined]
 in anonymous at ./<missing>:?
 in eval(::Module, ::Any) at ./boot.jl:234
 in eval(::Module, ::Any) at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 in process_options(::Base.JLOptions) at ./client.jl:242
 in _start() 

LoadError: Failed to precompile QuantEcon to /Users/Max/.julia/lib/v0.5/QuantEcon.ji.

In [25]:

ygrid0 = linspace(-4, 4, 10)
agrid0 = linspace(0.0.^0.4, 100.0.^0.4, 25).^(1/0.4)

# method one, using the Basis constructor multiple times
basis = Basis(SplineParams(agrid0, 0, 3),  # cubic spline
              SplineParams(ygrid0, 0, 1))  # linear

LoadError: UndefVarError: Basis not defined